# 금융 고객 컴플레인 데이터 유형분류 

## (1) 코랩에서 TPU를 선택
- Colab에서 런타임 > 런타임 유형 변경 > 하드웨어 가속기에서 'TPU'선택

In [2]:
# import tensorflow as tf
# import os

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

In [3]:
# strategy = tf.distribute.TPUStrategy(resolver)

## (2) 데이터 로드
- 금융 소비자 불만 유형(**"or"** 이 붙은 것은 유형이 확실하지 않을 것으로 판단)
1. Credit reporting, credit repair services, or other personal consumer reports
2. Debt collection(채권 추심)
3. Credit card or prepaid card(신용카드 or 선불카드) 
4. Mortgage(모기지)
5. Checking or savings account(당좌예금 or 저축예금)
6. Money transfer, virtual currency, or money service (송금, 가상화폐, 할부대출,...) 
7. Vehicle loan or lease (자동차 대출 or 리스)
8. Student loan(학자금 대출)
9. Payday loan, title loan, or personal loan (개인 융자)

In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.3 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 6.6 MB 47.7 MB/s 
     |████████████████████████████████| 596 kB 68.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import transformers
transformers.__version__

'4.20.1'

In [6]:
import re
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

In [7]:
path = '/content/drive/MyDrive/Section04_project_complaints_data/complaints_data.csv'
df = pd.read_csv(path)
df.head(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,07/22/20,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,Debt of XXXX was paid in XX/XX/2019 after movi...,None,"National Credit Systems,Inc.",GA,30311,Servicemember,Consent provided,Web,07/22/20,Closed with explanation,Yes,NaN,3758197
1,01/06/21,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I am stressed out that you have failed to main...,None,"EQUIFAX, INC.",FL,33322,None,Consent provided,Web,01/06/21,Closed with explanation,Yes,NaN,4051806
2,05/27/21,"Credit reporting, credit repair services, or o...",Credit reporting,Unable to get your credit report or credit score,Other problem getting your report or credit score,I have been waiting on hold for the past 2 day...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,AZ,85340,None,Consent provided,Web,05/28/21,Closed with explanation,Yes,NaN,4412191
3,05/19/21,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,I have try to settle this with USCB and they w...,None,"USCB, Inc.",CA,93309,None,Consent provided,Web,05/19/21,Closed with explanation,Yes,NaN,4390090
4,08/21/20,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,i summited an issue where an account was repor...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,19128,None,Consent provided,Web,08/21/20,Closed with explanation,Yes,NaN,3808488
5,01/05/21,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Investigation took more than 30 days,I have disputed the errors on my credit report...,None,"EQUIFAX, INC.",FL,33025,None,Consent provided,Web,01/05/21,Closed with explanation,Yes,NaN,4048439
6,06/15/20,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with fraud alerts or security freezes,None,On or around XX/XX/2020 I attempted to lift my...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NJ,08203,None,Consent provided,Web,06/15/20,Closed with explanation,Yes,NaN,3699259
7,08/20/20,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,I filed a dispute with the credit reporting ag...,None,"EQUIFAX, INC.",GA,30909,None,Consent provided,Web,08/20/20,Closed with explanation,Yes,NaN,3804605
8,01/06/21,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,On XXXX2020 I sent a letter regarding inaccura...,None,"EQUIFAX, INC.",GA,30060,None,Consent provided,Web,01/06/21,Closed with explanation,Yes,NaN,4049671
9,05/08/21,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I am not sure what happened to cause XXXX XXXX...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NV,89141,None,Consent provided,Web,05/08/21,Closed with explanation,Yes,NaN,4362426


In [8]:
## 불만 텍스트를 남긴 데이터 수
print("총 데이터 수:",len(df))

총 데이터 수: 502645


In [9]:
## 불만 유형
print("총 불만 유형: ",len(df['Product'].value_counts()))
print()
for i, category in enumerate(df['Product'].value_counts().index):
    print(f"{i+1}번 유형: {category}")    

총 불만 유형:  9

1번 유형: Credit reporting, credit repair services, or other personal consumer reports
2번 유형: Debt collection
3번 유형: Credit card or prepaid card
4번 유형: Mortgage
5번 유형: Checking or savings account
6번 유형: Money transfer, virtual currency, or money service
7번 유형: Vehicle loan or lease
8번 유형: Student loan
9번 유형: Payday loan, title loan, or personal loan


In [10]:
df = df[['Consumer complaint narrative', 'Product']]
df.head()

,Consumer complaint narrative,Product
0,Debt of XXXX was paid in XX/XX/2019 after movi...,Debt collection
1,I am stressed out that you have failed to main...,"Credit reporting, credit repair services, or o..."
2,I have been waiting on hold for the past 2 day...,"Credit reporting, credit repair services, or o..."
3,I have try to settle this with USCB and they w...,"Credit reporting, credit repair services, or o..."
4,i summited an issue where an account was repor...,"Credit reporting, credit repair services, or o..."


In [11]:
## columns명을 label로 변경
df.rename(columns={'Product':'label'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
df['label'].value_counts(normalize=True)

Credit reporting, credit repair services, or other personal consumer reports    0.524306
Debt collection                                                                 0.167745
Credit card or prepaid card                                                     0.091981
Mortgage                                                                        0.072747
Checking or savings account                                                     0.058182
Money transfer, virtual currency, or money service                              0.031088
Vehicle loan or lease                                                           0.021926
Student loan                                                                    0.017766
Payday loan, title loan, or personal loan                                       0.014259
Name: label, dtype: float64

## (3) 데이터 전처리

### 1) 대,소문자 변경 및 정규식 변환

In [13]:
## 소비자 텍스트 소문자로 일괄변경 및 정규표현식으로 전처리 함수 

def lower_and_regex(sentence):
    """
    모든 대문자를 소문자로 변경 후
    정규식을 이용하여 알파벳 소문자 이외의 구두점 및 쉼표 삭제
    """
    sentence = sentence.lower()
    sentence = re.sub(r"a-z", "", sentence)
    return sentence

In [14]:
## lower_and_regex 함수 적용 ##

df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(lower_and_regex)
print(df['Consumer complaint narrative'][0])
print()

debt of xxxx was paid in xx/xx/2019 after moving out of apartment in xx/xx/2019. the apartment agent failed to complete my new address with " xxxx '' so i never received the letter. 

i paid it under act number ending - xxxx but i received a mailing in xxxx under account number xxxx for the same exact amount. 

this company has taken it amongst themselves to reopen a paid balance under a new account number. this is fraudulent. i've called several times to get no assistance and only allowed the option to leave a voicemail.



### 2) label Text단어를 숫자형 라벨로 변경

-  Credit reporting, credit repair services, or other personal consumer reports -> 0번 인덱스
-  Debt collection(채권 추심) -> 1번 인덱스
-  Credit card or prepaid card(신용카드 or 선불카드) -> 2번 인덱스
-  Mortgage(모기지) -> 3번 인덱스
-  Checking or savings account(당좌예금 or 저축예금) -> 4번 인덱스
-  Money transfer, virtual currency, or money service (송금, 가상화폐, 할부대출,...) -> 5번 인덱스
- Vehicle loan or lease (자동차 대출 or 리스) -> 6번 인덱스
- Student loan(학자금 대출) -> 7번 인덱스
- Payday loan, title loan, or personal loan (개인 융자) -> 8번 인덱스

In [15]:
label_list = list(df['label'].value_counts().index)
for i in range(len(label_list)):
    df.loc[df['label']==label_list[i], 'label'] = i

In [16]:
df['label'].value_counts(normalize=True)

0    0.524306
1    0.167745
2    0.091981
3    0.072747
4    0.058182
5    0.031088
6    0.021926
7    0.017766
8    0.014259
Name: label, dtype: float64

In [17]:
df[:20]

,Consumer complaint narrative,label
0,debt of xxxx was paid in xx/xx/2019 after movi...,1
1,i am stressed out that you have failed to main...,0
2,i have been waiting on hold for the past 2 day...,0
3,i have try to settle this with uscb and they w...,0
4,i summited an issue where an account was repor...,0
5,i have disputed the errors on my credit report...,0
6,on or around xx/xx/2020 i attempted to lift my...,0
7,i filed a dispute with the credit reporting ag...,0
8,on xxxx2020 i sent a letter regarding inaccura...,0
9,i am not sure what happened to cause xxxx xxxx...,0


### 3) Train, Test 데이터 분류

In [18]:
#stratify=target값을 부여하여 class비율(ratio)을 train/validation에 유지

x_train, x_test, y_train, y_test = train_test_split(df['Consumer complaint narrative'], df['label'], test_size=0.2, shuffle=True, stratify=df['label'], random_state=42)

In [19]:
## 정상적으로 split이 되었는지 확인

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((402116,), (402116,), (100529,), (100529,))

### 4) 문장 토큰화 진행

In [20]:
## BERT 토크나이저 객체 생성

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [21]:
# 단어 토큰만 진행

print(tokenizer.tokenize(df['Consumer complaint narrative'][0]))

['debt', 'of', 'xx', '##x', '##x', 'was', 'paid', 'in', 'xx', '/', 'xx', '/', '2019', 'after', 'moving', 'out', 'of', 'apartment', 'in', 'xx', '/', 'xx', '/', '2019', '.', 'the', 'apartment', 'agent', 'failed', 'to', 'complete', 'my', 'new', 'address', 'with', '"', 'xx', '##x', '##x', "'", "'", 'so', 'i', 'never', 'received', 'the', 'letter', '.', 'i', 'paid', 'it', 'under', 'act', 'number', 'ending', '-', 'xx', '##x', '##x', 'but', 'i', 'received', 'a', 'mail', '##ing', 'in', 'xx', '##x', '##x', 'under', 'account', 'number', 'xx', '##x', '##x', 'for', 'the', 'same', 'exact', 'amount', '.', 'this', 'company', 'has', 'taken', 'it', 'amongst', 'themselves', 'to', 're', '##open', 'a', 'paid', 'balance', 'under', 'a', 'new', 'account', 'number', '.', 'this', 'is', 'fraud', '##ulent', '.', 'i', "'", 've', 'called', 'several', 'times', 'to', 'get', 'no', 'assistance', 'and', 'only', 'allowed', 'the', 'option', 'to', 'leave', 'a', 'voice', '##mail', '.']


In [22]:
# 단어 토큰화 이후 정수형으로 변환까지 인코딩

print(tokenizer.encode(df['Consumer complaint narrative'][0]))

[101, 55869, 10108, 76294, 10686, 10686, 10134, 25938, 10106, 76294, 120, 76294, 120, 19493, 10662, 23557, 10950, 10108, 54617, 10106, 76294, 120, 76294, 120, 19493, 119, 10105, 54617, 18980, 20775, 10114, 17876, 15127, 10751, 32198, 10169, 107, 76294, 10686, 10686, 112, 112, 10380, 177, 14794, 12296, 10105, 19310, 119, 177, 25938, 10271, 10571, 19833, 11487, 29077, 118, 76294, 10686, 10686, 10473, 177, 12296, 169, 30049, 10230, 10106, 76294, 10686, 10686, 10571, 23200, 11487, 76294, 10686, 10686, 10142, 10105, 11561, 45809, 24074, 119, 10531, 12100, 10393, 15109, 10271, 48593, 20182, 10114, 11639, 47656, 169, 25938, 40162, 10571, 169, 10751, 23200, 11487, 119, 10531, 10124, 93070, 91647, 119, 177, 112, 10323, 11552, 11736, 13465, 10114, 15329, 10192, 33514, 10111, 10893, 18162, 10105, 32562, 10114, 21964, 169, 21264, 85505, 119, 102]


In [23]:
## [CLS]는 분류문제를 풀기 위해 삽입된 특별 토큰
tokenizer.decode(tokenizer.encode(df['Consumer complaint narrative'][0]))

'[CLS] debt of xxxx was paid in xx / xx / 2019 after moving out of apartment in xx / xx / 2019. the apartment agent failed to complete my new address with " xxxx\'\' so i never received the letter. i paid it under act number ending - xxxx but i received a mailing in xxxx under account number xxxx for the same exact amount. this company has taken it amongst themselves to reopen a paid balance under a new account number. this is fraudulent. i\'ve called several times to get no assistance and only allowed the option to leave a voicemail. [SEP]'

In [24]:
for elem in tokenizer.encode(df['Consumer complaint narrative'][0]):
  print(tokenizer.decode(elem))

[ C L S ]
d e b t
o f
x x
# # x
# # x
w a s
p a i d
i n
x x
/
x x
/
2 0 1 9
a f t e r
m o v i n g
o u t
o f
a p a r t m e n t
i n
x x
/
x x
/
2 0 1 9
.
t h e
a p a r t m e n t
a g e n t
f a i l e d
t o
c o m p l e t e
m y
n e w
a d d r e s s
w i t h
"
x x
# # x
# # x
'
'
s o
i
n e v e r
r e c e i v e d
t h e
l e t t e r
.
i
p a i d
i t
u n d e r
a c t
n u m b e r
e n d i n g
-
x x
# # x
# # x
b u t
i
r e c e i v e d
a
m a i l
# # i n g
i n
x x
# # x
# # x
u n d e r
a c c o u n t
n u m b e r
x x
# # x
# # x
f o r
t h e
s a m e
e x a c t
a m o u n t
.
t h i s
c o m p a n y
h a s
t a k e n
i t
a m o n g s t
t h e m s e l v e s
t o
r e
# # o p e n
a
p a i d
b a l a n c e
u n d e r
a
n e w
a c c o u n t
n u m b e r
.
t h i s
i s
f r a u d
# # u l e n t
.
i
'
v e
c a l l e d
s e v e r a l
t i m e s
t o
g e t
n o
a s s i s t a n c e
a n d
o n l y
a l l o w e d
t h e
o p t i o n
t o
l e a v e
a
v o i c e
# # m a i l
.
[ S E P ]


In [25]:
print(tokenizer.decode(101))

[ C L S ]


In [26]:
print(tokenizer.decode(102))

[ S E P ]


In [27]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)

[CLS] : 101
[SEP] : 102


In [28]:
max_seq_len = 150

In [29]:
## tokenizer 및 pad_seqeunces 동시에 진행 ##
encoded_result = tokenizer.encode(df['Consumer complaint narrative'][0], max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길이 :', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[101, 55869, 10108, 76294, 10686, 10686, 10134, 25938, 10106, 76294, 120, 76294, 120, 19493, 10662, 23557, 10950, 10108, 54617, 10106, 76294, 120, 76294, 120, 19493, 119, 10105, 54617, 18980, 20775, 10114, 17876, 15127, 10751, 32198, 10169, 107, 76294, 10686, 10686, 112, 112, 10380, 177, 14794, 12296, 10105, 19310, 119, 177, 25938, 10271, 10571, 19833, 11487, 29077, 118, 76294, 10686, 10686, 10473, 177, 12296, 169, 30049, 10230, 10106, 76294, 10686, 10686, 10571, 23200, 11487, 76294, 10686, 10686, 10142, 10105, 11561, 45809, 24074, 119, 10531, 12100, 10393, 15109, 10271, 48593, 20182, 10114, 11639, 47656, 169, 25938, 40162, 10571, 169, 10751, 23200, 11487, 119, 10531, 10124, 93070, 91647, 119, 177, 112, 10323, 11552, 11736, 13465, 10114, 15329, 10192, 33514, 10111, 10893, 18162, 10105, 32562, 10114, 21964, 169, 21264, 85505, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길이 : 150


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [30]:
# 세그멘트 인풋
print([0]*max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [31]:
# 마스크 인풋
valid_num = len(tokenizer.encode(df['Consumer complaint narrative'][0]))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [32]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
       
        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
        # token_type_id는 세그먼트 임베딩을 위한 것으로 문장이 1개이므로 전부 0으로 통일.
        # token_type_ids는 세그먼트(segment)정보로 파인튜닝을 실시할 때는 모두 0을 줌
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [33]:
## Data로부터 텍스트를 입력 특징으로 변환
## 입력 문자열 분리하기
## [CLS]를 맨 앞에 삽입하기
## [SEP]를 첫번째 문장과 두 번째 문장 사이, 그리고 제일 마지막 위치에 넣기
## 토큰이 첫번째 문장에 속하는지 두번째 문장에 속하는지 알려주는 토큰 타입 id 생성

X_train, Y_train = convert_examples_to_features(x_train, y_train, max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/402116 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 402116/402116 [26:45<00:00, 250.54it/s]


In [34]:
X_test, Y_test = convert_examples_to_features(x_test, y_test, max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/100529 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 100529/100529 [06:36<00:00, 253.25it/s]


In [35]:
# 최대 길이: 150
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]
label = Y_train[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [  101 10105 44546 10146 11206 10146 10105 10106 72326 10107 17585   177
 10374 33338 10114 10149 10169   102     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0

### 4) 토큰화된 문장 길이 확인

In [36]:
# ## 토큰의 갯수 확인

# na_len = []

# for i in df['Consumer complaint narrative']:
#     na_len.append(len(df['Consumer complaint narrative'][i]))

# np.argmax(na_len) ## 인덱스 값은 반환

In [37]:
# ## 최대 문장 길이
# na_len[836237]

In [38]:
# print("최대 문장길이: ",np.max(na_len)) ## 최대길이
# print("평균 문장길이: ",np.mean(na_len)) ## 평균길이
# print("중앙값: ",np.median(na_len)) ## 중앙값

In [39]:
# na_len[0]

In [40]:
# len(df['Consumer complaint narrative'][0])

In [41]:
# import matplotlib.pyplot as plt
# plt.hist(na_len)
# plt.show()

## (4) 모델 생성

In [71]:
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [72]:
max_seq_len = 150

In [73]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [74]:
print(outputs)
print(outputs[0])
print(outputs[1])

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 150, 768) dtype=float32 (created by layer 'tf_bert_model_6')>, pooler_output=<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model_6')>, past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)
KerasTensor(type_spec=TensorSpec(shape=(None, 150, 768), dtype=tf.float32, name=None), name='tf_bert_model_6/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model_6'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model_6/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model_6'")


In [75]:
model.summary()

Model: "tf_bert_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
Total params: 177,853,440
Trainable params: 177,853,440
Non-trainable params: 0
_________________________________________________________________


In [76]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(9,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [77]:
model = TFBertForSequenceClassification("bert-base-multilingual-cased")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [78]:
model.fit(X_train, Y_train, epochs=2, batch_size=64, validation_split=0.2,verbose=2)

Epoch 1/2


ResourceExhaustedError: ignored

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=1024)
print("test loss: ", test_loss)
print("test loss: ", test_acc)